# Loading the Graph

In [1]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph.load("transformed_kg.json")
kg

KnowledgeGraph(nodes: 25, relationships: 92)

# Visualizing uisng Neo4j

No nodes getting added

In [ ]:
from neo4j import GraphDatabase

class Neo4jGraph:
    def __init__(self, uri, user, password):
        """
        Connects to the Neo4j database and creates necessary indexes.
        """
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.create_indexes()

    def close(self):
        """
        Closes the Neo4j session.
        """
        self.driver.close()

    def clear_graph(self):
        """
        Clears all nodes and relationships in the Neo4j database.
        """
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")

    def create_node(self, node):
        """
        Creates a node in Neo4j with specified properties.
        """
        with self.driver.session() as session:
            # Ensure only the desired properties are set for 'Code' nodes
            if node.type.value == "Code":
                props = {
                    "id": str(node.id),
                    "page_content": node.properties.get("page_content", ""),
                    "headlines": node.properties.get("headlines", "")
                }
                query = (
                    f"CREATE (n:{node.type.value} {{ "
                    f"id: $id, "
                    f"page_content: $page_content, "
                    f"headlines: $headlines "
                    f"}})"
                )
                session.run(query, **props)
            else:
                # Handle other node types if necessary
                pass

    def create_relationship(self, relationship):
        """
        Creates a relationship between two nodes in Neo4j with properties.
        If the relationship is bidirectional, two directed relationships are created.
        """
        with self.driver.session() as session:
            props = {
                "source_id": str(relationship.source.id),
                "target_id": str(relationship.target.id),
                "bidirectional": relationship.bidirectional
            }

            # Construct property string dynamically
            props_str = ", ".join(f"{k}: ${k}" for k in relationship.properties.keys())
            rel_props = f"{{ {props_str} }}" if props_str else ""

            if relationship.bidirectional:
                # Create two directed relationships to simulate bidirectionality
                query = (
                    f"MATCH (a {{id: $source_id}}), (b {{id: $target_id}}) "
                    f"CREATE (a)-[r:{relationship.type} {rel_props}]->(b), "
                    f"(b)-[r2:{relationship.type} {rel_props}]->(a)"
                )
            else:
                # Create a single directed relationship
                query = (
                    f"MATCH (a {{id: $source_id}}), (b {{id: $target_id}}) "
                    f"CREATE (a)-[r:{relationship.type} {rel_props}]->(b)"
                )

            session.run(query, source_id=props["source_id"], target_id=props["target_id"], **relationship.properties)

    def upload_knowledge_graph(self, kg):
        """
        Uploads the entire KnowledgeGraph (nodes and relationships) to Neo4j.
        """
        self.clear_graph()  # Clear existing graph before uploading

        print("Uploading nodes...")
        for node in kg.nodes:
            self.create_node(node)

        print("Uploading relationships...")
        for relationship in kg.relationships:
            self.create_relationship(relationship)

        print("Upload complete! Open Neo4j Browser to view the graph.")

    def create_indexes(self):    
        """
        Creates indexes on frequently queried node properties to improve query performance.
        """
        with self.driver.session() as session:
            session.run("CREATE INDEX IF NOT EXISTS FOR (n:Code) ON (n.id)")

            # If needed, add an index for other node types
            session.run("CREATE INDEX IF NOT EXISTS FOR (n:Code) ON (n.page_content)")
            session.run("CREATE INDEX IF NOT EXISTS FOR (n:Code) ON (n.headlines)")
            print("Indexes created successfully.")


In [ ]:
URI = ""
neo4j_user = "neo4j"
neo4j_password = ""
AUTH = (neo4j_user, neo4j_password)

In [ ]:
from neo4j import GraphDatabase

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [ ]:
neo4j_graph = Neo4jGraph(neo4j_uri, neo4j_user, neo4j_password)
neo4j_graph.upload_knowledge_graph(kg)
neo4j_graph.close()

# Visualizing using Pyvis

In [2]:
from pyvis.network import Network
net = Network()

In [ ]:
for i in kg.nodes:
	label = str(i.id)
	title = i.properties.get("page_content")
	if i.type == "document":
		source = i.properties.get("document_metadata").get("source")[11:-3]
		label  += f"document:{source}"
	net.add_node(n_id= str(i.id),label=label, title=title)

In [4]:
for i in kg.relationships:
	source = str(i.source.id)
	target = str(i.target.id)
	rel = i.type
	label=i.type, 
	net.add_edge(source=source,to=target, title=rel, label=label, arrows='to')

In [5]:
import webbrowser
import tempfile

temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".html")
net.show(temp_file.name, notebook=False)
webbrowser.open(temp_file.name)

/var/folders/n7/chh0c4z50hq_7kc4xkf5b5fw0000gn/T/tmpfnfuzh_o.html


True